# Degree Heating Week calculation 

Using this notebook to compute DHW diagnostics and output annual maximum DHW 

Applied to the OFAM future simulation with statics compute for various GWLs

Did the calculations with megamem


In [1]:
import xarray as xr
from dask.distributed import Client
import dask.array as da
from dask import delayed
import numpy as np
import os
import matplotlib.pyplot as plt
from glob import glob
import warnings

import dask
# Set configuration options
dask.config.set({
    'distributed.comm.timeouts.connect': '90s',  # Timeout for connecting to a worker
    'distributed.comm.timeouts.tcp': '90s',  # Timeout for TCP communications
})

In [2]:
import dask

# Set configuration options
dask.config.set({
    'distributed.comm.timeouts.connect': '90s',  # Timeout for connecting to a worker
    'distributed.comm.timeouts.tcp': '90s',  # Timeout for TCP communications
})

client = Client()
client

2024-07-01 22:40:07,425 - distributed.preloading - INFO - Creating preload: /g/data/hh5/public/apps/dask-optimiser/schedplugin.py
2024-07-01 22:40:07,431 - distributed.utils - INFO - Reload module schedplugin from .py file
2024-07-01 22:40:07,456 - distributed.preloading - INFO - Import preload module: /g/data/hh5/public/apps/dask-optimiser/schedplugin.py


Modifying workers


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /node/gadi-mmem-bdw-0003.gadi.nci.org.au/3770/proxy/8787/status,
Dashboard: /node/gadi-mmem-bdw-0003.gadi.nci.org.au/3770/proxy/8787/status,Workers: 32
Total threads: 32,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37389,Workers: 32
Dashboard: /node/gadi-mmem-bdw-0003.gadi.nci.org.au/3770/proxy/8787/status,Total threads: 32
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:33181,Total threads: 1
Dashboard: /node/gadi-mmem-bdw-0003.gadi.nci.org.au/3770/proxy/33981/status,Memory: 0 B
Nanny: tcp://127.0.0.1:37147,


2024-07-01 22:48:53,370 - bokeh.server.protocol_handler - ERROR - error handling message
 message: Message 'PATCH-DOC' content: {'events': [{'kind': 'MessageSent', 'msg_type': 'bokeh_event', 'msg_data': {'type': 'event', 'name': 'document_ready', 'values': {'type': 'map'}}}]} 
 error: AssertionError()
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/bokeh/server/protocol_handler.py", line 97, in handle
    work = await handler(message, connection)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/bokeh/server/session.py", line 295, in patch
    return connection.session._handle_patch(message, connection)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/bokeh/server/connection.py", line 65, in session
    assert self._session is not None
AssertionError
2024-07-01 22:48:53,432 - tornado.application - ERROR - Uncaught excepti

In [3]:
def drop_stuff(ds, coords_to_drop, vars_to_drop):
    ds = ds.drop_vars(coords_to_drop, errors='ignore')
    ds = ds.drop_vars(vars_to_drop, errors='ignore')
    return ds

In [4]:
def load_data(coords_to_drop, vars_to_drop, dir1_new, dir2_new):
    pattern0 = sorted(glob(dir1_new + 'ocean_temp_sfc*.nc'))
    pattern1 = sorted(glob(dir1_new + 'ocean_temp_sfc*200[0-5]*.nc'))
    pattern2 = sorted(glob(dir2_new + 'ocean_temp_sfc*.nc'))

# store historical and future into dsst2
    dsst1 = xr.open_mfdataset(pattern0, parallel=True, preprocess=lambda x: drop_stuff(x, coords_to_drop, vars_to_drop)).squeeze()
    dsst2 = xr.open_mfdataset(pattern1 + pattern2, parallel=True, preprocess=lambda x: drop_stuff(x, coords_to_drop, vars_to_drop)).squeeze()

    sst = dsst2.temp
    sst = sst.rename({"Time": "time"})

    ssth= dsst1.temp
    ssth= ssth.rename({"Time": "time"})
    
    return sst, ssth

In [5]:
def process_mmm(ds, time_dim, start, end, variable, period):
# Compute the climatological monthly maximum value
    # Rechunk the data along the time dimension
    data = getattr(ds, variable).sel(**{time_dim: slice(start, end)}).persist()

    # Debug: Print the shape and chunking of the data
    print(f"Processing period {period},{start},{end} data shape: {data.shape}, chunks: {data.chunks}")

    # Calculate the 90th percentile
    mmm1 = data.resample(Time='1M').mean()
    mmm = mmm1.max(dim=time_dim)

    # Define the output file path
    file = f'/g/data/ia39/ncra/ocean/mhw/{variable}_mmm_{period}.nc'
    mmm.to_netcdf(file, compute=True)

    return mmm

# Read in existing Climatological monthly maximum
def read_mmm(period):
#    period='current' 'GW1p2'
    variable='temp'
    file =  f'/g/data/ia39/ncra/ocean/mhw/{variable}_mmm_{period}.nc'
    mmm= xr.open_dataset(file)['temp']
    return mmm

# Calculate mmm
def calc_mmm(dsst2,GWL_periods,period):
    (start,end)=GWL_periods[period]
    mmm= process_mmm(dsst2, 'Time', start, end, 'temp', period)
    return
    

In [6]:
%%time
# Degree Heating Week Calculate 
def calculate_dhw(sst, mmm1, period, start, end):
    sst1 = sst.sel(time=slice(start, end))
#    mmmp1= mmm1 +1  # monthly mean maximum + 1
# Compute hotspots and mask values less than 1
    sst_hotspot = sst1 - mmm1 # p1
    hmask = sst_hotspot >= 1  
    dhd = (sst_hotspot * hmask).persist()
# Calculate the cumulative sum over a 12-week period to get DHD
# convert from degree-days to degree-weeks
    dhw = (dhd.rolling(time=84, center=False).sum() * 1/7).persist()
# compute annual maximum of dhw 
    ymax=dhw.groupby('time.year').max('time').persist()
    ymax_max=ymax.max('year')   # max of the annual dhw
    ymax_min=ymax.min('year')   # min of the annual dhw
    dhw = dhw.rename('dhw').assign_attrs(units='degrees C week')
    ymax = ymax.rename('dhw_amax').assign_attrs(units='degrees C week')
    dhd = dhd.rename('dhd').assign_attrs(units='degrees C day')
    ymax_max = ymax_max.rename('dhw_max').assign_attrs(units='degrees C week')
    ymax_min = ymax_min.rename('dhw_min').assign_attrs(units='degrees C week')

#    result = xr.merge([sst_hotspot,dhd,dhw,ymax])
    result = xr.merge([ymax,ymax_max,ymax_min])

    return result
    


CPU times: user 3 µs, sys: 14 µs, total: 17 µs
Wall time: 37.7 µs


In [7]:
def save_results(result_ds, period):
    file_path = f"/g/data/ia39/ncra/ocean/{period}_dhw.nc"
    result_ds.to_netcdf(file_path)
    print(f"Saved results to {file_path}")

In [8]:
def process_all_periods(GWL_periods, sst, ssth):
    for period, (start, end) in GWL_periods.items():
        astr=f'DHW compute {period}'
        print(astr)
        if period == 'current':
            mmm = read_mmm('current')
            mmm.load()
            result_ds = calculate_dhw(ssth, mmm, period, start, end).compute()
        else:
            mmm = read_mmm('GW1p2')
            mmm.load()
            result_ds = calculate_dhw(sst, mmm, period, start, end).compute()
            
        save_results(result_ds, period)

In [ ]:
def main():
    warnings.filterwarnings('ignore')

    coords_to_drop = ['st_edges_ocean', 'nv', 'st_ocean']
    vars_to_drop = ['Time_bounds', 'average_DT', 'average_T1', 'average_T2', 'st_ocean']

    dir1_new = '/g/data/fp2/OFAM3/jra55_historical.1/surface/'
    dir2_new = '/g/data/fp2/OFAM3/jra55_rcp8p5/surface/'

    GWL_periods = {
        'current': ('1995-01-01', '2014-12-31'),
        'GW1p2': ('2001-01-01', '2020-12-31'),
        'GW1p5': ('2015-01-01', '2034-12-31'),
        'GW2p0': ('2030-01-01', '2049-12-31'),
        'GW3p0': ('2053-01-01', '2072-12-31'),
        'GW4p0': ('2074-01-01', '2093-12-31')
    }
#    GWL_periods = {
#        'GW4p0': ('2074-01-01', '2093-12-31') }
    
    print('load data')
    sst, ssth = load_data(coords_to_drop, vars_to_drop, dir1_new, dir2_new)

    print('process all periods',GWL_periods)
    process_all_periods(GWL_periods, sst, ssth)

#    client.close()

if __name__ == "__main__":
    main()

load data
process all periods {'current': ('1995-01-01', '2014-12-31'), 'GW1p2': ('2001-01-01', '2020-12-31'), 'GW1p5': ('2015-01-01', '2034-12-31'), 'GW2p0': ('2030-01-01', '2049-12-31'), 'GW3p0': ('2053-01-01', '2072-12-31'), 'GW4p0': ('2074-01-01', '2093-12-31')}
DHW compute current
Saved results to /g/data/ia39/ncra/ocean/current_dhw.nc
DHW compute GW1p2
Saved results to /g/data/ia39/ncra/ocean/GW1p2_dhw.nc
DHW compute GW1p5
Saved results to /g/data/ia39/ncra/ocean/GW1p5_dhw.nc
DHW compute GW2p0
Saved results to /g/data/ia39/ncra/ocean/GW2p0_dhw.nc
DHW compute GW3p0
Saved results to /g/data/ia39/ncra/ocean/GW3p0_dhw.nc
DHW compute GW4p0
